In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import datetime
import os

import xgboost as xgb
from sklearn.model_selection import train_test_split
import lightgbm
import warnings
from sklearn.metrics import mean_squared_error
from fbprophet import Prophet
warnings.filterwarnings('ignore')
#한글 깨짐방지
plt.rc('font',family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

C:\Users\User\Anaconda3\lib\site-packages\distributed\config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


In [2]:
path = './data/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission  = pd.read_csv(path + 'submission.csv')
merge_df = pd.concat([train,test])

In [3]:
train['DT'] = pd.to_datetime(train['date']+" "+train['time'])

In [4]:
train.head()

,store_id,date,time,card_id,amount,installments,days_of_week,holyday,DT
0,0,2016-12-14,18:05:31,d297bba73f,5,NaN,2,0,2016-12-14 18:05:31
1,0,2016-12-14,18:05:54,d297bba73f,-5,NaN,2,0,2016-12-14 18:05:54
2,0,2016-12-19,12:42:31,0880849c05,144,NaN,0,0,2016-12-19 12:42:31
3,0,2016-12-19,12:48:08,8b4f9e0e95,66,NaN,0,0,2016-12-19 12:48:08
4,0,2016-12-19,13:31:08,7ad237eed0,24,NaN,0,0,2016-12-19 13:31:08


In [5]:
d_sum = train.groupby(['date','store_id'])['amount'].sum().reset_index()
d_sum[d_sum['amount']<0]

,date,store_id,amount
34,2016-08-01,138,-180
1504,2016-08-04,1478,-11250
5810,2016-08-15,1683,-35
5823,2016-08-15,1753,-75
5881,2016-08-16,216,-1295
...,...,...,...
473957,2018-07-25,36,-500
476119,2018-07-27,513,-300
478867,2018-07-30,956,-295
479213,2018-07-30,1669,-250


In [6]:
train[(train.store_id == 1478)&(train.date == '2016-08-04')]

,store_id,date,time,card_id,amount,installments,days_of_week,holyday,DT
2785057,1478,2016-08-04,19:37:01,8589f5980e,-11250,NaN,3,0,2016-08-04 19:37:01


In [28]:
minus_amount_df = train[train.amount<0][['DT','store_id','card_id','amount']]
plus_amount_df = train[train.amount>0][['DT','store_id','card_id','amount']]

In [29]:
plus_index = list(plus_amount_df.index)

In [39]:
err_dic = dict()
non_match = []
double = []
for i in tqdm(range(minus_amount_df.shape[0])):
    minus_index = minus_amount_df.iloc[i].name
    minus_card = minus_amount_df.iloc[i].card_id 
    minus_store = minus_amount_df.iloc[i].store_id
    minus_DT = minus_amount_df.iloc[i].DT
    minus_amount = minus_amount_df.iloc[i].amount

    #store_id 와 card_id가 같으면서 날짜가 이전인경우 (각 조건을 나눠서 하는게 속도가 빠름)
    range_df = plus_amount_df[plus_amount_df.store_id == minus_store]
    range_df = range_df[range_df.card_id == minus_card]
    range_df = range_df[range_df.DT <= minus_DT]
#     range_df = plus_amount_df[(plus_amount_df.store_id == minus_store)&(plus_amount_df.card_id == minus_card)&(plus_amount_df.DT <= minus_DT)]
    #내가 해놓고 내가 이해가 안되는 지경 2020-07-01 17:54:21 
    
    '''
    range_df는 음수의 이전날짜면서, 상점명, 카드명이 동일한 경우
    target은 금액이 더 큰경우
    
    '''
    try:
        target = range_df[range_df.amount.apply(lambda x : x >= abs(minus_amount))]
        if len(target)<1: #금액이 큰게 없는경우 
            non_match.append(minus_index)
            pass

        if target.amount.isin([abs(minus_amount)]).sum()>0: #동일한 금액이 있는 경우 가장 최근날짜로 
            error_index = target[(target.amount == abs(minus_amount))].index.max()
        else :
            error_index = target.index.max() #동일한 금액이 없다면 그냥 최근 날짜로 (금액불일치)
    except :
        print("뭐지????????????????????",minus_index)
        pass
    
#     plus_amount_df.loc[error_index,'amount'] =+ minus_amount
        
    try : 
        if err_dic[error_index] < 0:
            err_dic[error_index] += minus_amount
            print(minus_index,error_index)
            double.append(error_index)
    except:
            err_dic[error_index] = minus_amount
# plus_amount_df.loc[plus_amount_df['index'].isin(order_dic.keys()),'amount'] += list(order_dic.values())

16048 16046
81063 81058
81064 81058
92481 92479
92836 92822
154872 154868
154877 154868
154878 154868
164843 164841
176596 176593
195660 195654
225140 225138
225555 225553
227676 227674
271809 271807
뭐지?? 400700
400700 400583
401224 401222
407416 407414
410174 410172
416456 416454
425266 425264
427196 427194



KeyboardInterrupt: 

In [47]:
non_match

[400700]

In [48]:
double

[16046,
 81058,
 81058,
 92479,
 92822,
 154868,
 154868,
 154868,
 164841,
 176593,
 195654,
 225138,
 225553,
 227674,
 271807,
 400583,
 401222,
 407414,
 410172,
 416454,
 425264,
 427194]

In [294]:
len(target)

0

In [289]:
sample = plus_amount_df.iloc[:10]

In [290]:
sample.drop(0)

,DT,store_id,card_id,amount
2,2016-12-19 12:42:31,0,0880849c05,144
3,2016-12-19 12:48:08,0,8b4f9e0e95,66
4,2016-12-19 13:31:08,0,7ad237eed0,24
5,2016-12-19 13:36:39,0,30b1422f77,44
6,2016-12-19 14:00:23,0,6dd8dad5ab,146
7,2016-12-19 14:01:36,0,a4a425c5d2,6
8,2016-12-19 14:09:46,0,f213c011fe,306
9,2016-12-19 14:13:02,0,a185b7b11f,51
10,2016-12-19 14:51:28,0,5af79433af,26


In [246]:
minus_amount

-12

In [251]:
minus_DT

Timestamp('2017-05-13 13:26:21')

In [250]:
target.amount.isin([abs(minus_amount)]).sum()

4

In [253]:
target[target.amount.isin([abs(minus_amount)])]

,DT,store_id,card_id,amount
110456,2017-03-27 15:53:32,23,93159945b3,12
110457,2017-03-27 15:56:02,23,93159945b3,12
110769,2017-04-17 14:54:57,23,93159945b3,12
110984,2017-05-01 14:51:28,23,93159945b3,12


In [249]:
target

,DT,store_id,card_id,amount
110452,2017-03-27 14:14:22,23,93159945b3,17
110456,2017-03-27 15:53:32,23,93159945b3,12
110457,2017-03-27 15:56:02,23,93159945b3,12
110769,2017-04-17 14:54:57,23,93159945b3,12
110872,2017-04-24 13:39:38,23,93159945b3,14
110874,2017-04-24 13:44:06,23,93159945b3,16
110984,2017-05-01 14:51:28,23,93159945b3,12
111051,2017-05-08 13:50:12,23,93159945b3,30


### 검증

In [41]:
i = 400700

In [42]:
check_store = minus_amount_df.loc[i]['store_id']
check_card = minus_amount_df.loc[i]['card_id']
check_amount = minus_amount_df.loc[i]['amount']
check_DT = minus_amount_df.loc[i]['DT']
minus_amount_df.loc[i]

DT          2016-08-01 17:56:07
store_id                    138
card_id              65be030b0f
amount                     -180
Name: 400700, dtype: object

In [43]:
minus_amount_df[(minus_amount_df.store_id == check_store)&(minus_amount_df.card_id == check_card)]

,DT,store_id,card_id,amount
400700,2016-08-01 17:56:07,138,65be030b0f,-180


In [44]:
plus_amount_df[(plus_amount_df.store_id == check_store)&(plus_amount_df.card_id == check_card)]

,DT,store_id,card_id,amount


In [278]:
range_df = plus_amount_df[plus_amount_df.store_id == check_store]
range_df = range_df[range_df.card_id == check_card]
range_df = range_df[range_df.DT <= check_DT]
#     range_df = plus_amount_df[(plus_amount_df.store_id == minus_store)&(plus_amount_df.card_id == minus_card)&(plus_amount_df.DT <= minus_DT)]

target = range_df[range_df.amount.apply(lambda x : x >= abs(minus_amount))]

In [274]:
target[target.amount.isin([abs(check_amount)])]

,DT,store_id,card_id,amount


In [270]:
target[target.amount.isin([abs(check_amount)])].index.max()

82421